In [86]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from gensim.models import KeyedVectors
from nltk.corpus import stopwords

# Load the text file
with open('data.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [90]:
text

"Yapay zeka, hayatımıza girdiği günden beri özellikle kendi sektörümde bazı meslek gruplarının yaptığı işleri büyük oranda yapabildiğini izliyorum. Visual designer, concept artist, game artist, illustrator, copywriter, UX designer, UI designer ve daha niceleri…Yapay zekanın neleri değiştirebileceğini ve hangi meslekleri nasıl tehdit ettiğini evdeki koltuğunuzda sorgulamaya hazırsanız başlayalım…\n\nHiçbir şey olmaz mı dersiniz ?\nBugün yeni GPT-4o’ya sesli olarak bana logaritma öğretmesini söyledim. Bana öğrenci kulislerindeki anlamlı soru ile yani logaritmanın güncel hayatta ne işimize yarayacağını anlatarak başladı. Benim bilgiyi edinmek istememdeki temel amacımı öğrendi ve log 2 (8)’in aslında ikinin kaç üstü 8'e eşit olması gerektiğini örneği ile temel mantığı basitçe anlattı. Benim amacımın bir sınava girmek olduğunu öğrenince bu sınav ile alakalı tüm (ÖSYM) veritabanını tarayıp çıkabilir soru tarzlarını analiz etti. Bilginin ne kadarını öğrenerek bu sınavlarda büyük oranda başarı

In [91]:
nltk.download('punkt')
nltk.download('stopwords')

# Split the text into paragraphs
paragraphs = text.split('\n\n')

# Preprocess text
stop_words = set(stopwords.words('turkish'))

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return words

def preprocess_documents(documents):
    return [' '.join(preprocess(doc)) for doc in documents]

all_processed_docs = preprocess_documents(paragraphs)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BatuhanYILMAZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BatuhanYILMAZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [92]:
all_processed_docs

['yapay zeka hayatımıza girdiği günden beri özellikle kendi sektörümde meslek gruplarının yaptığı işleri büyük oranda yapabildiğini izliyorum visual designer concept artist game artist illustrator copywriter ux designer ui designer niceleriyapay zekanın neleri değiştirebileceğini hangi meslekleri tehdit ettiğini evdeki koltuğunuzda sorgulamaya hazırsanız başlayalım',
 'hiçbir olmaz dersiniz bugün yeni gpt4oya sesli olarak bana logaritma öğretmesini söyledim bana öğrenci kulislerindeki anlamlı soru logaritmanın güncel hayatta işimize yarayacağını anlatarak başladı benim bilgiyi edinmek istememdeki temel amacımı öğrendi log 2 8in ikinin kaç üstü 8e eşit olması gerektiğini örneği temel mantığı basitçe anlattı benim amacımın bir sınava girmek olduğunu öğrenince sınav alakalı ösym veritabanını tarayıp çıkabilir soru tarzlarını analiz etti bilginin kadarını öğrenerek sınavlarda büyük oranda başarılı olabileceğimi düşünerek bana buna uygun örnek sorular hazırladı',
 'senaryo dijital ürün tasa

In [96]:
# TF-IDF Vectorizer
cv = CountVectorizer()
word_count_vector = cv.fit_transform(all_processed_docs)
tfidf_transformer = TfidfTransformer(smooth_idf=True, norm='l2', use_idf=True)
tfidf_transformer.fit(word_count_vector)
tf_idf_vector = tfidf_transformer.transform(word_count_vector)
W = tf_idf_vector.toarray()
W

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.12060472, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [105]:
# Load pre-trained Word2Vec model
word_vectors = KeyedVectors.load_word2vec_format('trmodel', binary=True)

In [106]:
word_vectors.most_similar(positive=["kral","kadın"],negative=["erkek"])

[('kraliçe', 0.508816123008728),
 ('kralı', 0.4718089699745178),
 ('kralın', 0.44512391090393066),
 ('kraliçesi', 0.4190150201320648),
 ('prenses', 0.4071306586265564),
 ('hükümdar', 0.4056030511856079),
 ('prens', 0.3978961408138275),
 ('kraliçenin', 0.39632880687713623),
 ('veliaht', 0.38219931721687317),
 ('tahtı', 0.3773398697376251)]

In [107]:
# Function to expand query using Word2Vec
def expand_query(query, top_n=2):
    processed_query = preprocess(query)
    expanded_query = processed_query.copy()
    for word in processed_query:
        if word in word_vectors.key_to_index:
            similar_words = word_vectors.most_similar(positive=[word], topn=top_n)
            expanded_query.extend([word[0] for word in similar_words])
    return ' '.join(expanded_query)

In [108]:
# Check expand_query function
test_data = "hayat ve ölüm"
expand_query(test_data)

'hayat ölüm yaşam yaşantı ölümün öldürülme'

In [109]:
# Load pre-trained BERTurk tokenizer and model
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = BertModel.from_pretrained("dbmdz/bert-base-turkish-cased")

In [110]:
# Function to get BERT embeddings
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Get the [CLS] token embedding
    return cls_embedding.squeeze().detach().numpy()

# Precompute BERT embeddings for all documents
bert_doc_vecs = np.array([get_bert_embedding(doc) for doc in all_processed_docs])

In [111]:
# Function to respond to query
def respond(query, n_best=5):
    # Expand the query using Word2Vec
    expanded_query = expand_query(query)
    print(f"Expanded query: {expanded_query}\n")
    
    # Convert the expanded query to TF-IDF vector
    expanded_query_vec = cv.transform([expanded_query])
    tf_idf_query = tfidf_transformer.transform(expanded_query_vec)
    
    # Generate BERT embedding for the original query (not expanded)
    bert_query_vec = get_bert_embedding(query)
    
    # Ensure the shapes are correct for cosine similarity
    bert_query_vec = bert_query_vec.reshape(1, -1)
    
    # Compute cosine similarity for TF-IDF
    tfidf_similarities = cosine_similarity(tf_idf_query, W)
    
    # Compute cosine similarity for BERT
    bert_similarities = cosine_similarity(bert_query_vec, bert_doc_vecs)
    
    # Combine similarities
    combined_similarities = (tfidf_similarities + bert_similarities) / 2
    
    # Rank documents based on the combined similarities
    sorted_doc_indices = np.argsort(combined_similarities[0])[::-1]
    
    # Print the sorted documents and their similarity scores
    for idx in sorted_doc_indices[:n_best]:
        print(f"{paragraphs[idx]} with similarity score: {combined_similarities[0][idx]:.4f}\n")

In [112]:
query = 'yapay zeka ve sağlık'
respond(query)

Expanded query: yapay zeka sağlık suni zekâ zekâ zekanın saglık sağlik

Komplocu bunlar 1: Diyelim ki > bir ülkenin sağlık bakanlığı özel bir şirketin geliştirdiği yapay zeka destekli bir sağlık asistanını ve hastanesini akredite olarak tanımış olsun. Bu durumda bu kurum insanlara bir doktor yerine kendi sağlık asistanına tedavi olması gerektiğini ikna etmek için ne gibi değer önerileri sunabilirdi? with similarity score: 0.2007

İşte ütopik sağlık şirketimizin asistanına tedavi olmanı için önerilerimiz: 7/24 hızlı müdahale imkanı sağlar. Sürekli olarak en son tıbbi araştırmaları ve bulguları analiz eder, tüm branşların bilgisine sahip olup konsültasyon ihtiyacını ortadan kaldırır. İnsan faktöründen kaynaklanan hata oranını düşürür, büyük veri analizi ve geçmiş vakalardan elde edilen bilgilerle doğru teşhis ve tedavi imkanı sunar. Hastaların geçmiş sağlık verilerini kullanarak kişiselleştirilmiş tedavi planları oluşturur ve maliyetleri düşürerek daha fazla kişiye ulaşılabilir hale geti

In [113]:
query = 'risk altındaki meslekler'
respond(query)

Expanded query: risk altındaki meslekler riskler riskleri altında altındadır meslekleri rasyolar

Yapay zeka, hayatımıza girdiği günden beri özellikle kendi sektörümde bazı meslek gruplarının yaptığı işleri büyük oranda yapabildiğini izliyorum. Visual designer, concept artist, game artist, illustrator, copywriter, UX designer, UI designer ve daha niceleri…Yapay zekanın neleri değiştirebileceğini ve hangi meslekleri nasıl tehdit ettiğini evdeki koltuğunuzda sorgulamaya hazırsanız başlayalım… with similarity score: 0.0903

Sanki ikna olmuş mu gibisiniz? Komplocu bunlar 2: Kanayan yaramız; Adalet Sistemine yapay zeka entegresi güzellemeleri; Sabah akşam mecliste tartışılan anayasa değişikliği oturumlarından anlıyoruz ki değişimine çok ta karşı çıkamayacağımız alanlardan biri adalet sistemi gibi geliyor.. Türkiye’de her yıl ortalama olarak 7 milyon dava açılıyor hakim başına 671 dava düşüyor ve sistemin yavaşlığı mağduriyetleri büyütüyor. Bu bağlamda yapay zekanın yargılama süreçlerine ent

In [114]:
query = 'riskli meslek'
respond(query)

Expanded query: riskli meslek masraflı zahmetli mesleki hemşirelik

Yapay zeka, hayatımıza girdiği günden beri özellikle kendi sektörümde bazı meslek gruplarının yaptığı işleri büyük oranda yapabildiğini izliyorum. Visual designer, concept artist, game artist, illustrator, copywriter, UX designer, UI designer ve daha niceleri…Yapay zekanın neleri değiştirebileceğini ve hangi meslekleri nasıl tehdit ettiğini evdeki koltuğunuzda sorgulamaya hazırsanız başlayalım… with similarity score: 0.0584

Daha uç bir örnek için; Genel cerrah bir arkadaşımla bir apandisit hastasının teşhisi konusunda yaptığımız deneyde, yapay zekanın doktorun gözden kaçırabileceği birçok noktayı atlamayan uzman bir yaklaşım sunduğunu gördük. Önce hastanın şikayetlerini ve temel bilgilerini yapay zekaya ilettik. Yapay zeka, adım adım ihtimalleri elemek için sorular sordu. Örneğin, 24 yaşındaki kadın hastanın adet sancısı ya da yerleşme sancısı çekebileceği ihtimalini değerlendirdi. Daha sonra fiziki muayene için hasta

In [115]:
query = 'ürkünç senaryo'
respond(query)

Expanded query: ürkünç senaryo Warrenton opelousas senaryoyu senaryosu

Senaryo o ya belki dijital ürün tasarımının yapay zeka ile tasarlandığı, yapay zeka tarafından geliştirilen, içerik pazarlamasına uygun blog yazılı ve görsel içerikleri, onboarding içerikleri, yardım ekranları yine yapay zeka tarafından hazırlanan ironik olsun diye 24 Kasım’da yayınlanacak olan “Yapay Öğretmenim” adlı bir uygulama olacağını düşünelim. Apple Search Ads uzmanı gibi kendi uygulamasının pazarlamasını da nasıl yapması gerektiğini bilen yapay zekamızın bu uygulaması kaç kez indirilir dersiniz? with similarity score: 0.0388

Daha uç bir örnek için; Genel cerrah bir arkadaşımla bir apandisit hastasının teşhisi konusunda yaptığımız deneyde, yapay zekanın doktorun gözden kaçırabileceği birçok noktayı atlamayan uzman bir yaklaşım sunduğunu gördük. Önce hastanın şikayetlerini ve temel bilgilerini yapay zekaya ilettik. Yapay zeka, adım adım ihtimalleri elemek için sorular sordu. Örneğin, 24 yaşındaki kadın hast